In [37]:
from __future__ import print_function
import keras
import numpy as np
from tensorflow.keras.datasets import cifar10, cifar100
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Layer
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop
import tensorflow.keras.backend as K
import tensorflow as tf
from datetime import datetime
import os

In [2]:
print(keras.__version__)

2.2.4


In [14]:
class MAct(Layer):

    def __init__(self, **kwargs):
        super(MAct, self).__init__(**kwargs)
        self.supports_masking = True

    def build(self, input_shape):
        self.c = self.add_weight(name="c",
                                shape=(input_shape[1],),
                                initializer='ones',
                                trainable=True) # Initialiseerida c ühtedeks / nullideks
        self.b = self.add_weight(name="b",
                                shape=(input_shape[1],),
                                initializer='ones',
                                trainable=True) # Initialiseerida b nullideks
        super(MAct, self).build(input_shape)
        

    def call(self, inputs):
        first_exp = tf.exp(self.c - tf.square(inputs))

        p = (first_exp + tf.exp(self.b)) / tf.reduce_sum(first_exp + tf.exp(self.b), axis=1, keepdims=True)
        
        #p = tf.exp(inputs) / tf.reduce_sum(tf.exp(inputs), axis=0, keepdims=True)
        return p

    def compute_output_shape(self, input_shape):
        return input_shape

In [15]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model_MAct_smaller.h5'

In [16]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [17]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
#model.add(Activation('softmax'))
model.add(MAct())

# initiate RMSprop optimizer
opt = RMSprop(learning_rate=0.0001, decay=1e-6)

In [18]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [19]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [20]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=1)

Using real-time data augmentation.
Epoch 1/100
1563/1563 [==============================] - 206s 132ms/step - loss: 2.1353 - accuracy: 0.2514 - val_loss: 2.0508 - val_accuracy: 0.3194
Epoch 2/100
1563/1563 [==============================] - 210s 135ms/step - loss: 2.0304 - accuracy: 0.3552 - val_loss: 1.9777 - val_accuracy: 0.3874
Epoch 3/100
1563/1563 [==============================] - 217s 139ms/step - loss: 1.9661 - accuracy: 0.4023 - val_loss: 1.9029 - val_accuracy: 0.4762
Epoch 4/100
1563/1563 [==============================] - 211s 135ms/step - loss: 1.9046 - accuracy: 0.4447 - val_loss: 1.8407 - val_accuracy: 0.5048
Epoch 5/100
1563/1563 [==============================] - 210s 134ms/step - loss: 1.8402 - accuracy: 0.4840 - val_loss: 1.7676 - val_accuracy: 0.5383
Epoch 6/100
1563/1563 [==============================] - 208s 133ms/step - loss: 1.7772 - accuracy: 0.5034 - val_loss: 1.7255 - val_accuracy: 0.5241
Epoch 7/100
1563/1563 [==============================] - 220s 141ms/ste

1563/1563 [==============================] - 208s 133ms/step - loss: 1.0853 - accuracy: 0.6405 - val_loss: 0.9984 - val_accuracy: 0.6591
Epoch 56/100
1563/1563 [==============================] - 206s 132ms/step - loss: 1.0880 - accuracy: 0.6391 - val_loss: 1.0356 - val_accuracy: 0.6452
Epoch 57/100
1563/1563 [==============================] - 216s 138ms/step - loss: 1.0802 - accuracy: 0.6407 - val_loss: 1.0375 - val_accuracy: 0.6524
Epoch 58/100
1563/1563 [==============================] - 220s 141ms/step - loss: 1.0831 - accuracy: 0.6415 - val_loss: 1.1012 - val_accuracy: 0.6083
Epoch 59/100
1563/1563 [==============================] - 225s 144ms/step - loss: 1.0878 - accuracy: 0.6372 - val_loss: 1.1028 - val_accuracy: 0.6274
Epoch 60/100
1563/1563 [==============================] - 214s 137ms/step - loss: 1.0821 - accuracy: 0.6406 - val_loss: 1.0204 - val_accuracy: 0.6492
Epoch 61/100
1563/1563 [==============================] - 211s 135ms/step - loss: 1.0792 - accuracy: 0.6415 - val

In [21]:
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at /Users/rain.vagel/confidence-thesis/saved_models/keras_cifar10_trained_model_MAct_smaller.h5 


# CIFAR-10 Keras Baseline

In [11]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Test loss: 0.7422410913467408
Test accuracy: 0.7515


# CIFAR-10 MAct

In [22]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Test loss: 1.064668455219269
Test accuracy: 0.6465


In [13]:
# Score trained model. lrate 0.00001 MAct
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Test loss: 1.4918159454345703
Test accuracy: 0.5759


In [24]:
loaded_model = tf.keras.models.load_model('saved_models/keras_cifar10_trained_model.h5')

In [43]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
num_classes = 100

In [44]:
y_test = keras.utils.to_categorical(y_test, num_classes)
x_test = x_test.astype('float32')
x_test /= 255

In [47]:
y_test.shape

(10000, 100)

In [48]:
preds = loaded_model.predict(x_test,verbose=0)


In [49]:
print(preds)

[[8.18924308e-01 1.99347036e-03 1.04623213e-01 ... 1.29628126e-02
  4.63963039e-02 5.56130754e-03]
 [3.02304537e-03 1.12224836e-03 1.00136824e-01 ... 2.20916513e-02
  1.37565413e-03 2.34888028e-03]
 [4.30083536e-02 6.42834522e-04 1.27081543e-01 ... 1.93030328e-01
  1.13379536e-02 3.64030502e-03]
 ...
 [3.60678695e-02 1.86410807e-02 1.30608514e-01 ... 7.03652576e-02
  1.02320816e-02 2.86830962e-02]
 [3.85492131e-05 7.27888983e-05 4.69965627e-03 ... 4.58995899e-04
  3.34104661e-05 1.73419612e-04]
 [2.98651345e-02 9.55347493e-02 3.17417681e-02 ... 3.61595154e-02
  3.21442746e-02 4.34607774e-01]]


In [50]:
max_conf = np.max(preds,1)

In [53]:
print(np.mean(max_conf))

0.5041824


In [54]:
loaded_model = tf.keras.models.load_model('saved_models/keras_cifar10_trained_model_MAct.h5')
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
num_classes = 100
y_test = keras.utils.to_categorical(y_test, num_classes)
x_test = x_test.astype('float32')
x_test /= 255
preds = loaded_model.predict(x_test,verbose=0)
max_conf = np.max(preds,1)
print(np.mean(max_conf))

ValueError: Unknown layer: MAct